In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import json_lines
import preprocess_temp as P
import model.parsers as M

### Load Data

In [3]:
directory = './conala-corpus/'
train_file = directory + 'train.json'
test_file = directory + 'test.json'

with open(train_file) as f:
    train_data = json.load(f)
    
with open(test_file) as f:
    test_data = json.load(f)

In [4]:
mine_file = directory + 'mined.jsonl'
mine_data = []
with open(mine_file) as f:
    mine_data = [line for line in json_lines.reader(f)]

### Let's preprocess the data. Everything is in Preprocess.py
### Adding mined data

In [5]:
mine_data[0]

{'id': '34705205_34705233_0',
 'intent': 'Sort a nested list by two elements',
 'parent_answer_post_id': 34705233,
 'prob': 0.8690001442846342,
 'question_id': 34705205,
 'snippet': 'sorted(l, key=lambda x: (-int(x[1]), x[0]))'}

In [6]:
# intent processing includes lowercase, remove punctuation'?'
train_intent, train_codes = P.process_data(train_data)
test_intent, test_codes = P.process_data(test_data)

In [7]:
mine_intent, mine_codes = P.process_data(mine_data, mine=True)

In [8]:
# this class is used for code2actions and actions2code
ast_action = P.Ast_Action()

In [9]:
train_actions = []

for code in train_codes:
    train_actions.append(ast_action.code2actions(code))

In [10]:
word_lst = P.vocab_list(train_intent, cut_freq=2)
act_lst, token_lst = P.action_list(train_actions, cut_freq=0)

In [11]:
word2num = dict(zip(word_lst, range(0,len(word_lst))))
act2num = dict(zip(act_lst, range(0,len(act_lst))))
token2num = dict(zip(token_lst, range(0,len(token_lst))))

In [12]:
train_loader = P.get_train_loader(train_intent, train_actions, word2num, act2num, token2num, batch_size=48)

In [13]:
test_loader = P.get_test_loader(test_intent, word2num, batch_size=1)

In [14]:
action_index_copy = act2num[P.GenTokenAction('copy')]
action_index_gen = act2num[P.GenTokenAction('token')]

### Model

In [15]:
import torch
import time

In [16]:
from collections import namedtuple
hyperParamMap = {
    #### General configuration ####
    'cuda': True,      # Use gpu
    'mode': 'train',   # train or test

    #### Embedding sizes ####
    'embed_size': 128,         # Size of word embeddings
    'action_embed_size': 128,  # Size of ApplyRule/GenToken action embeddings
    'field_embed_size': 64,    # Embedding size of ASDL fields
    'type_embed_size': 64,     # Embeddings ASDL types

    #### Decoding sizes ####
    'hidden_size': 256,        # Size of LSTM hidden states
    'beam_size': 5,
    
    #### training schedule details ####
    'valid_metric': 'acc',                # Metric used for validation
    'valid_every_epoch': 1,               # Perform validation every x epoch
    'log_every': 10,                      # Log training statistics every n iterations
    'save_to': 'model',                   # Save trained model to
    'clip_grad': 5.,                      # Clip gradients
    'max_epoch': 10,                      # Maximum number of training epoches
    'optimizer': 'Adam',                  # optimizer
    'lr': 0.001,                          # Learning rate
    'lr_decay': 0.5,                      # decay learning rate if the validation performance drops
    'verbose': False,                     # Verbose mode

    #### decoding/validation/testing ####
    'load_model': None,                   # Load a pre-trained model
    'beam_size': 1,                       # Beam size for beam search
    'decode_max_time_step': 100,          # Maximum number of time steps used in decoding and sampling
    'sample_size': 5,                     # Sample size
    'test_file': '',                      # Path to the test file
    'save_decode_to': None,               # Save decoding results to file
}

HyperParams = namedtuple('HyperParams', list(hyperParamMap.keys()), verbose=False)
hyperParams = HyperParams(**hyperParamMap)

In [17]:
model = M.Model(hyperParams, action_size=len(act_lst), token_size=len(token_lst), word_size=len(word_lst), 
                      action_index_copy=action_index_copy, action_index_gen=action_index_gen)

In [18]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0008)
lossFunc = torch.nn.CrossEntropyLoss()

In [30]:
model.train()
epoch_begin = time.time()
for e in range(20):
    for batch_ind, x in enumerate(train_loader):
        optimizer.zero_grad()

        (action_logits, action_labels), (copy_logits, copy_labels), (token_logits, token_labels) = model(x)

        loss1 = lossFunc(action_logits, action_labels)
        loss2 = torch.DoubleTensor([0.0])
        if len(copy_logits) > 0:
            loss2 = lossFunc(copy_logits, copy_labels)
        loss3 = torch.DoubleTensor([0.0])
        if len(token_logits) > 0:
            loss3 = lossFunc(token_logits, token_labels)

        total_loss = loss1 + loss2.double() + loss3.double()
        total_loss.backward()

        # clip gradient
        if hyperParams.clip_grad > 0.:
            grad_norm = torch.nn.utils.clip_grad_norm_(model.parameters(), hyperParams.clip_grad)

        optimizer.step()

        if batch_ind % hyperParams.log_every == hyperParams.log_every - 1:
            print('--------------------------epoch {} batch {}-----------------------------'.format(e, batch_ind))
            print("Action loss: {}".format(loss1.data))
            print("Copy loss: {}".format(loss2.data))
            print("Token loss: {}".format(loss3.data))
            report_loss = report_examples = 0.

    print('epoch elapsed %ds' % (time.time() - epoch_begin))

--------------------------epoch 0 batch 9-----------------------------
Action loss: 1.50036199387705
Copy loss: 1.8467381000518799
Token loss: 3.1456639766693115


--------------------------epoch 0 batch 19-----------------------------
Action loss: 1.0444523776391839
Copy loss: 1.9628452062606812
Token loss: 3.1182303428649902


--------------------------epoch 0 batch 29-----------------------------
Action loss: 0.8577129307601006
Copy loss: 1.737626314163208
Token loss: 2.6550490856170654


--------------------------epoch 0 batch 39-----------------------------
Action loss: 1.0485215867465225
Copy loss: 1.6723734140396118
Token loss: 2.976733922958374


--------------------------epoch 0 batch 49-----------------------------
Action loss: 0.7515159954857923
Copy loss: 1.8979887962341309
Token loss: 3.025411367416382
epoch elapsed 109s


...

--------------------------epoch 3 batch 19-----------------------------
Action loss: 0.6579105984644791
Copy loss: 1.7333481311798096
Token loss: 1.6888489723205566


--------------------------epoch 3 batch 29-----------------------------
Action loss: 0.5792926479354329
Copy loss: 1.5967553853988647
Token loss: 1.689185380935669


--------------------------epoch 3 batch 39-----------------------------
Action loss: 0.6706192528132491
Copy loss: 1.7725468873977661
Token loss: 2.0128355026245117


--------------------------epoch 3 batch 49-----------------------------
Action loss: 0.5443874418566484
Copy loss: 1.648914098739624
Token loss: 2.1709280014038086
epoch elapsed 437s


KeyboardInterrupt: 

In [31]:
torch.save((model).state_dict(), 'Parameters/cut_2_0_v3.t7')

In [29]:
model.load_state_dict(torch.load('Parameters/cut_2_0_v2.t7'))

In [36]:
sample_sent, sample_sent_txt = next(iter(test_loader))
print(sample_sent)
print(sample_sent_txt)
sample_hypothesis = model.parse(sample_sent, sample_sent_txt, act_lst, token_lst, ast_action, decode_method='greedy')
ast_action.actions2code(sample_hypothesis.actions)

[tensor([1291,    3, 1411, 1411,    8,   19,  252, 1070])]
[['send', 'a', 'signal', '`signal.SIGUSR1`', 'to', 'the', 'current', 'process']]


"os.system('%Y-%m-%d -c')"

In [161]:
print(sample_hypothesis.actions)
ast_action.actions2code(sample_hypothesis.actions)

[ApplyRule[stmt -> Expr(expr value)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[any], ApplyRule[expr -> GeneratorExp(expr elt, comprehension* generators)], ApplyRule[expr -> Compare(expr left, cmpop* ops, expr* comparators)], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[isinstance], ApplyRule[expr -> Name(identifier id)], GenToken[x], Reduce, Reduce, Reduce, Reduce, ApplyRule[comprehension -> comprehension(expr target, expr iter, expr* ifs)], ApplyRule[expr -> Name(identifier id)], GenToken[i], ApplyRule[expr -> Call(expr func, expr* args, keyword* keywords)], ApplyRule[expr -> Name(identifier id)], GenToken[range], ApplyRule[expr -> Num(object n)], GenToken[100], ApplyRule[expr -> Num(object n)], GenToken[9], Reduce, Reduce, Reduce, Reduce, Reduce, Reduce]
Expr(value=Call(func=Name(id='any'), args=[GeneratorExp(elt=Compare(left=Call(func=Name

IndexError: list index out of range

In [25]:
ground_truth = []
for e in test_data:
    ground_truth.append(e['snippet'])
# print(ground_truth)

In [38]:
import json

ground_truth = []
for e in test_data:
    ground_truth.append(e['snippet'])

# model.eval()
cnt = 1
code_list = []
import time
import dataset.util as U
# import dataset.bleu_score as bleu
# import nltk.translate as bleu
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
t0 = time.time()
test_loader = P.get_test_loader(test_intent, word2num, batch_size=1)
for ind, (sample_sent, sample_sent_txt) in enumerate(test_loader):
    t1 = time.time()
    sample_hypothesis = model.parse(sample_sent, sample_sent_txt, act_lst, token_lst, ast_action, decode_method='greedy')
    try:
        code = ast_action.actions2code(sample_hypothesis.actions)
    except:
        code = ' '
    code_list.append(code)
    t2 = time.time()
    print('----------------------answer {}, this one cost {}s, total cost {}s'.format(cnt, t2 - t1, t2 - t0))
    print('my code:   ', code)
    print('ground truth:   ', ground_truth[ind])
    code_token_list = U.tokenize_for_bleu_eval(code)
    target_token_list = U.tokenize_for_bleu_eval(ground_truth[ind])

    # First argument should be list of list of words from ground truth (in our case only one ground truth)
    # Second argument should be our prediction, and it's a list of word
    bleu = sentence_bleu([target_token_list],
                             code_token_list)
    print('bleu socre {}'.format(bleu))
    cnt = cnt + 1
print(code_list)
with open('./answer.txt', 'w') as outfile:
    json.dump(code_list, outfile, indent=1)


----------------------answer 1, this one cost 0.0313870906829834s, total cost 0.034013986587524414s
my code:    os.system('%Y-%m-%d -c')
ground truth:    os.kill(os.getpid(), signal.SIGUSR1)
bleu socre 0.34823528327578546
----------------------answer 2, this one cost 0.025599956512451172s, total cost 0.06061291694641113s
my code:    webbrowser.listdir('log', 'libs')
ground truth:    bytes.fromhex('4a4b4c').decode('utf-8')
bleu socre 0.37621186824601177
----------------------answer 3, this one cost 0.017236948013305664s, total cost 0.0785069465637207s
my code:    map(lst)
ground truth:    all(x == myList[0] for x in myList)
bleu socre 0.04186576731652839
----------------------answer 4, this one cost 0.04041790962219238s, total cost 0.1196129322052002s
my code:    re.findall('filename.png', 'D:\\testdir', '', 'title')
ground truth:    print('%*s : %*s' % (20, 'Python', 20, 'Very Good'))
bleu socre 0.5191064695319341
----------------------answer 5, this one cost 0.021172285079956055s, tot

/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 3-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)
/anaconda3/lib/python3.6/site-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 4-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


----------------------answer 8, this one cost 0.07846307754516602s, total cost 0.29548096656799316s
my code:    subprocess.set_title('script2.py', '$%s', 'neededArgumetGoHere', '+20')
ground truth:    subprocess.check_output('ps -ef | grep something | wc -l', shell=True)
bleu socre 0.408248290463863
----------------------answer 9, this one cost 0.049948930740356445s, total cost 0.3466529846191406s
my code:    a.concatenate('b')
ground truth:    """""".join(['a', 'b', 'c'])
bleu socre 0.06928592765229934
----------------------answer 10, this one cost 0.03168296813964844s, total cost 0.3792989253997803s
my code:    np.einsum('ij,kj->jik', 'b')
ground truth:    pd.Series(list(set(s1).intersection(set(s2))))
bleu socre 0.5432852079582625
----------------------answer 11, this one cost 0.027704715728759766s, total cost 0.4080348014831543s
my code:    ax.set_xlabel('Temperature')
ground truth:    client.send('HTTP/1.0 200 OK\r\n')
bleu socre 0.186245115152549
----------------------answer 12, 

----------------------answer 14, this one cost 0.06520891189575195s, total cost 0.5516018867492676s
my code:    re.findall('filename.png', '*', 'this')
ground truth:    ' a \n b \r\n c '.split('\n')
bleu socre 0.43749609513070276
----------------------answer 15, this one cost 0.040075063705444336s, total cost 0.593019962310791s
my code:    concatenate(concatenate(zip(b)))
ground truth:    """:""".join(str(x) for x in b)
bleu socre 0.1973862244299071
----------------------answer 16, this one cost 0.03248095512390137s, total cost 0.6264278888702393s
my code:    re.findall('_(?:for|or|and)_', 'sad_pandas_and_happy_cats_for_people', 'Python'
    )
ground truth:    Entry.objects.filter()[:1].get()
bleu socre 0.6930977286178778
----------------------answer 17, this one cost 0.028824806213378906s, total cost 0.6562888622283936s
my code:    sorted(zip(arr))
ground truth:    a.sum(axis=1)
bleu socre 0.6337834876616586
----------------------answer 18, this one cost 0.018526077270507812s, total c

----------------------answer 21, this one cost 0.03899502754211426s, total cost 0.7797119617462158s
my code:    re.findall("//a[starts-with(text(),'some", 'http://example.org/#comments', '')
ground truth:    time.strptime('30/03/09 16:31:32.123', '%d/%m/%y %H:%M:%S.%f')
bleu socre 0.1670910486608537
----------------------answer 22, this one cost 0.033843040466308594s, total cost 0.8148839473724365s
my code:    s.import_module('11B', 'a.b')
ground truth:    my_float = float(my_string.replace(',', ''))
bleu socre 0.3016006458695292
----------------------answer 23, this one cost 0.042076826095581055s, total cost 0.8578839302062988s
my code:    .findall('11B', '')
ground truth:    float('123,456.908'.replace(',', ''))
bleu socre 0.17185045819660794
----------------------answer 24, this one cost 0.022456884384155273s, total cost 0.8812918663024902s
my code:    plt.info('log')
ground truth:    sys.path.append('/path/to/whatever')
bleu socre 0.2709850435615371
----------------------answer 25,

...

----------------------answer 466, this one cost 0.03261089324951172s, total cost 15.83645486831665s
my code:    calendar.writerows('property', 0)
ground truth:    [x for x in sents if not x.startswith('@$\t') and not x.startswith('#')]
bleu socre 0.06570416643305552
----------------------answer 467, this one cost 0.029220104217529297s, total cost 15.86686897277832s
my code:    int(set(range(i)))
ground truth:    Entry.objects.filter(pub_date__contains='08:00')
bleu socre 0.4482700320176827
----------------------answer 468, this one cost 0.034796953201293945s, total cost 15.902431011199951s
my code:    sorted(sorted(operator.items()))
ground truth:    list.sort(key=lambda item: (item['points'], item['time']))
bleu socre 0.1293170825812159
----------------------answer 469, this one cost 0.024409055709838867s, total cost 15.927579879760742s
my code:    logging.read_csv('INSERT', '')
ground truth:    (t - datetime.datetime(1970, 1, 1)).total_seconds()
bleu socre 0.41095816241285266
-------

----------------------answer 474, this one cost 0.03076314926147461s, total cost 16.061962127685547s
my code:    ax.execute('INSERT INTO table +0900')
ground truth:    df.to_csv(filename, index=False)
bleu socre 0.7071067811865476
----------------------answer 475, this one cost 0.03206205368041992s, total cost 16.09471893310547s
my code:    ax.find_element_by_xpath('INSERT', 'Basic')
ground truth:    json_data = json.loads(unescaped)
bleu socre 0.7071067811865476
----------------------answer 476, this one cost 0.054662227630615234s, total cost 16.15027904510498s
my code:    s.split('s', '', '', '', ',', ',', ',', ',')
ground truth:    [chr(i) for i in range(127)]
bleu socre 0.49616830003403634
----------------------answer 477, this one cost 0.024213075637817383s, total cost 16.17548704147339s
my code:    re.compile('chapter3', 'BDE')
ground truth:    newFile.write(struct.pack('5B', *newFileBytes))
bleu socre 0.30641737018051174
----------------------answer 478, this one cost 0.03513383

----------------------answer 481, this one cost 0.03186202049255371s, total cost 16.28509283065796s
my code:    os.system('%Y-%m-%d -c')
ground truth:    f = open('myfile', 'w')
f.write('hi there\n')
f.close()
bleu socre 0.2292681346193333
----------------------answer 482, this one cost 0.02463388442993164s, total cost 16.31064796447754s
my code:    os.system('words.txt')
ground truth:    with open('somefile.txt', 'a') as the_file:
    the_file.write('Hello\n')
bleu socre 0.08965720846107784
----------------------answer 483, this one cost 0.0212862491607666s, total cost 16.332874059677124s
my code:    s.rfind().&
ground truth:    s.encode('iso-8859-15')
bleu socre 0.2719520463054227
----------------------answer 484, this one cost 0.02698802947998047s, total cost 16.360591888427734s
my code:    plt.find_elements_by_xpath("//*[contains(text(), 'My Button')]")
ground truth:    AuthorizedEmail.objects.filter(group=group).order_by('-added')[0]
bleu socre 0.4342906676853412
-----------------

----------------------answer 489, this one cost 0.029705286026000977s, total cost 16.503178119659424s
my code:    writer.writerow('...j,...j->...', 0)
ground truth:    list(cf.get_range().get_keys())
bleu socre 0.645777692487244
----------------------answer 490, this one cost 0.023797035217285156s, total cost 16.527695894241333s
my code:    datetime.strftime('%B')
ground truth:    datetime.datetime.now()
bleu socre 0.48549177170732344
----------------------answer 491, this one cost 0.030673742294311523s, total cost 16.55910086631775s
my code:    [i for i in i]
ground truth:    next(i for i, x in enumerate(lst) if not isinstance(x, bool) and x == 1)
bleu socre 0.0292697063277546
----------------------answer 492, this one cost 0.023020029067993164s, total cost 16.583109855651855s
my code:    a.a('a')
ground truth:    a[:] = [(x - 13) for x in a]
bleu socre 0.30934850332660563
----------------------answer 493, this one cost 0.03533220291137695s, total cost 16.619271993637085s
my code:    

----------------------answer 495, this one cost 0.27468323707580566s, total cost 16.91757607460022s
my code:    None.()
ground truth:    re.findall('"(http.*?)"', s, re.MULTILINE | re.DOTALL)
bleu socre 0.0048833661205058675
----------------------answer 496, this one cost 0.032440185546875s, total cost 16.95079207420349s
my code:    re.sub('#(\\w+)', '<markup>\\1</markup>', 'this')
ground truth:    re.findall('http://[^t][^s"]+\\.html', document)
bleu socre 0.4766389972493081
----------------------answer 497, this one cost 0.03391098976135254s, total cost 16.985734939575195s
my code:    re.sub(',', '', '', '')
ground truth:    mystring.replace(' ', '! !').split('!')
bleu socre 0.1818483989940587
----------------------answer 498, this one cost 0.02161383628845215s, total cost 17.008195877075195s
my code:    os.Popen('file.txt', 'file2.txt')
ground truth:    open(path, 'r')
bleu socre 0.45180100180492244
----------------------answer 499, this one cost 0.02698206901550293s, total cost 17.

In [40]:
outfile.close()

In [42]:
code_list

["os.system('%Y-%m-%d -c')",
 "webbrowser.listdir('log', 'libs')",
 'map(lst)',
 "re.findall('filename.png', 'D:\\\\testdir', '', 'title')",
 "re.replace('')",
 'dict(list(d.values()))',
 'dict(list(d.values()))',
 "subprocess.set_title('script2.py', '$%s', 'neededArgumetGoHere', '+20')",
 "a.concatenate('b')",
 "np.einsum('ij,kj->jik', 'b')",
 "ax.set_xlabel('Temperature')",
 "plt.parse('a', '')",
 "re.findall('{http://www.w3.org/2002/07/owl#}Class', '3.1415', '')",
 "re.findall('filename.png', '*', 'this')",
 'concatenate(concatenate(zip(b)))',
 "re.findall('_(?:for|or|and)_', 'sad_pandas_and_happy_cats_for_people', 'Python'\n    )",
 'sorted(zip(arr))',
 "dparser.urldefrag('monkey')",
 'items(set(a))',
 "subprocess.system('grep -c', 'rt', 'some.file')",
 're.findall("//a[starts-with(text(),\'some", \'http://example.org/#comments\', \'\')',
 "s.import_module('11B', 'a.b')",
 ".findall('11B', '')",
 "plt.info('log')",
 "re.findall('#(\\\\w+)', '\\\\\\\\\\\\g<0>')",
 "os.system('%Y-%m-